In [15]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import lil_matrix
import joblib


In [20]:
df = pd.read_csv("dfListo.csv")
df.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,id_collection,name_collection,id_genres,name_genres,id_production,name_production,id_language,name_language,id_countrie,name_countrie
0,30000000.0,862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,10194.0,Toy Story Collection,"16.0,35.0,10751.0","Animation,Comedy,Family",3.0,Pixar Animation Studios,en,English,US,United States of America
1,65000000.0,8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,NaN,NaN,"12.0,14.0,10751.0","Adventure,Fantasy,Family","559.0,2550.0,10201.0","TriStar Pictures,Teitler Film,Interscope Commu...","en,fr","English,Français",US,United States of America
2,0.0,15602.0,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,119050.0,Grumpy Old Men Collection,"10749.0,35.0","Romance,Comedy","6194.0,19464.0","Warner Bros.,Lancaster Gate",en,English,US,United States of America
3,16000000.0,31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,NaN,NaN,"35.0,18.0,10749.0","Comedy,Drama,Romance",306.0,Twentieth Century Fox Film Corporation,en,English,US,United States of America
4,0.0,11862.0,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,96871.0,Father of the Bride Collection,35.0,Comedy,"5842.0,9195.0","Sandollar Productions,Touchstone Pictures",en,English,US,United States of America


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45376 non-null  float64
 1   id                 45376 non-null  float64
 2   original_language  45365 non-null  object 
 3   overview           44435 non-null  object 
 4   popularity         45376 non-null  float64
 5   release_date       45376 non-null  object 
 6   revenue            45376 non-null  float64
 7   runtime            45130 non-null  float64
 8   status             45296 non-null  object 
 9   tagline            20398 non-null  object 
 10  title              45376 non-null  object 
 11  vote_average       45376 non-null  float64
 12  return             45376 non-null  float64
 13  id_collection      4488 non-null   float64
 14  name_collection    4488 non-null   object 
 15  id_genres          42992 non-null  object 
 16  name_genres        429

In [10]:
df["title"].sample(45)

14865                                         The Crazies
14761                                  Four Eyed Monsters
5538                                             The Ring
5159                                       The Cat's Meow
36313                             EK Chotti Si Love Story
6162                                       Fahrenheit 451
7690                                              College
31123                              A Killer Among Friends
32363                                      Happiness Runs
7663                                While the City Sleeps
15887                                               Devil
20667                                 A Bunch of Amateurs
42213                                   Dibu: La Película
16186                          Last of the Red Hot Lovers
37165                                Oysters at Nam Kee's
19150                                            The Firm
8817                        Merry Christmas, Mr. Lawrence
27681         

In [17]:
# df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d', errors='coerce')

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45464 entries, 0 to 45463
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45376 non-null  float64
 1   id                 45376 non-null  float64
 2   original_language  45365 non-null  object 
 3   overview           44435 non-null  object 
 4   popularity         45376 non-null  float64
 5   release_date       45376 non-null  object 
 6   revenue            45376 non-null  float64
 7   runtime            45130 non-null  float64
 8   status             45296 non-null  object 
 9   tagline            20398 non-null  object 
 10  title              45376 non-null  object 
 11  vote_average       45376 non-null  float64
 12  return             45376 non-null  float64
 13  id_collection      4488 non-null   float64
 14  name_collection    4488 non-null   object 
 15  id_genres          42992 non-null  object 
 16  name_genres        429

In [22]:
def peliculas_mes(mes):
    df['release_date'] = pd.to_datetime(df['release_date'])
    
    if type(mes) == str:
        meses = {"enero": 1, "febrero": 2, "marzo": 3, "abril": 4, "mayo": 5, "junio": 6, 
                 "julio": 7, "agosto": 8, "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12}
        mes = mes.lower()
        if mes not in meses:
            raise ValueError("El mes ingresado no es válido.")
        mes = meses[mes]
    
    peliculas_mes = df[df['release_date'].dt.month == mes]
    cantidad = len(peliculas_mes)
    respuesta = {'mes': mes, 'cantidad': cantidad}
    return respuesta

resultado = peliculas_mes("diciembre")
print(resultado)

{'mes': 12, 'cantidad': 3786}


In [29]:
def peliculas_dia(dia):
    df['release_date'] = pd.to_datetime(df['release_date'])
    peliculas_dia = df[df['release_date'].dt.day == dia]
    cantidad = len(peliculas_dia)
    respuesta = {'dia': dia, 'cantidad': cantidad}
    return respuesta

resultado = peliculas_dia(1)
print(resultado)

{'dia': 1, 'cantidad': 5838}


In [30]:
def franquicia(franquicia):
    peliculas_franquicia = df[df['name_collection'] == franquicia]
    cantidad = len(peliculas_franquicia)
    ganancia_total = peliculas_franquicia['return'].sum()
    ganancia_promedio = peliculas_franquicia['return'].mean()
    respuesta = {'franquicia': franquicia, 'cantidad': cantidad, 'ganancia_total': ganancia_total, 'ganancia_promedio': ganancia_promedio}
    return respuesta

resultado = franquicia('Toy Story Collection')
print(resultado)

{'franquicia': 'Toy Story Collection', 'cantidad': 3, 'ganancia_total': 12.4518011, 'ganancia_promedio': 4.150600366666667}


In [31]:
def peliculas_pais(pais):
    peliculas_pais = df[df['name_countrie'] == pais]
    cantidad = len(peliculas_pais)
    respuesta = {'pais': pais, 'cantidad': cantidad}
    return respuesta

resultado = peliculas_pais("United States of America")
print(resultado)

{'pais': 'United States of America', 'cantidad': 17846}


In [32]:
def productoras(productora):
    # Filtrar el DataFrame por el nombre de la productora
    df_productora = df[df['name_production'] == productora]
    ganancia_total = df_productora['return'].sum()
    cantidad = df_productora['id'].count()
    respuesta = {'productora': productora, 'ganancia_total': ganancia_total, 'cantidad': cantidad}
    
    return respuesta

resultado = productoras("Pixar Animation Studios")
print(resultado)

{'productora': 'Pixar Animation Studios', 'ganancia_total': 75.39816266666666, 'cantidad': 28}


In [30]:
def retorno(titulo):
    pelicula = df[df['title'] == titulo]
    inversion = pelicula['budget'].iloc[0]
    ganancia = pelicula['revenue'].iloc[0]
    retorno = (ganancia - inversion) / inversion if inversion != 0 else 0
    anio = int(pelicula['release_date'].iloc[0].strftime("%Y"))
    respuesta = {'pelicula': titulo, 'inversion': inversion, 'ganancia': ganancia, 'retorno': retorno, 'anio': anio}
    return respuesta

resultado = retorno("Toy Story")
print(resultado)

{'pelicula': 'Toy Story', 'inversion': 30000000.0, 'ganancia': 373554033.0, 'retorno': 11.4518011, 'anio': 1995}
